In [1]:
from imp import reload
import SubRecommender as sr
import pandas as pd
import json

In [2]:
reload(sr)

<module 'SubRecommender' from 'C:\\Users\\macle\\Desktop\\UPC Masters\\Semester 2\\CI\\SubRecommender\\SubRecommender.py'>

In [3]:
tst = sr.SubRecommender(train_data_file="data/train_reddit_data.json",save_model_file="models/tst",sequence_chunk_size=51)

In [4]:
tst.train_network(num_epochs=9)

AttributeError: 'SubRecommender' object has no attribute 'training_labels'

In [ ]:
print(len(tst.training_labels))

In [ ]:
with open('data/train_reddit_data.json','r') as data_file:    
    reddit_data = json.load(data_file)

In [ ]:
df = pd.DataFrame(reddit_data,columns=['user','subreddit','utc_stamp'])
df['utc_stamp'] = pd.to_datetime(df['utc_stamp'],unit='s')
df.sort_values(by=['user','utc_stamp'], ascending=True, inplace=True)
users = list(df.groupby('user')['user'].nunique().keys())
sub_list = list(df.groupby('subreddit')['subreddit'].nunique().keys())

In [ ]:
all_recs = []
for usr in users[0:25]:
    user_comment_subs = list(df.loc[df['user'] == usr]['subreddit'].values)
    rec_subs = tst.rec_subs(user_comment_subs)
    all_recs.append(rec_subs[0])

In [ ]:
all_recs